# Add Dependences

In [88]:
include("../../src/forces_func.jl")
include("../../src/struct_data.jl")
include("../../src/test.jl")
include("../../src/forces.jl")
include("../../src/neighbor.jl")
include("../../src/run_event.jl")

@make_struct_func Cubic

force_func (generic function with 1 method)

# Add Data

In [89]:
parameters = Model(
    Cubic(
        μ₁      = 0.05,
        rₘᵢₙ    = 2,
        rₘₐₓ    = 3
    ),
    Contractile(
        fₚ       = 0.1
    ),
    Time(
        t_f      = 100000, 
        dt       = 0.1
    ),
    Neighbor(
        n_knn    = 100,
        nn       = 100
    ),
    Geometry(
        R_agg    = 9,
        position = [
            0.0    0.0    0.0;
            0.0    0.0    0.0
        ]
    ),
    Simulation(
        n_text   = 200,
        path     = "Fusion.xyz"
    )
)

## Var before func
println("\n Loading Force Function")
CUDA.@time force(r) =  force_func(parameters.Force::LennardJones, r)


 Loading Force Function
  0.000028 seconds (21 CPU allocations: 1.628 KiB)


force (generic function with 1 method)

## Starting Data

In [90]:
# Loading Data
# CUDA.@time X = Float32.(readdlm("$(Int(parameters.Geometry.R_agg)).xyz")[3:end,2:end]) |> cu
CUDA.@time  X = Float32.(readdlm("../../data/init/Sphere/$(Int(parameters.Geometry.R_agg)).xyz")[3:end,2:end]) |> cu

  0.021998 seconds (8.06 k CPU allocations: 317.672 KiB) (1 GPU allocation: 6.305 KiB, 0.06% memmgmt time)


538×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -1.5  -4.62  -7.35
  0.5  -4.62  -7.35
 -2.5  -2.89  -7.35
 -0.5  -2.89  -7.35
  1.5  -2.89  -7.35
  3.5  -2.89  -7.35
 -3.5  -1.15  -7.35
 -1.5  -1.15  -7.35
  0.5  -1.15  -7.35
  2.5  -1.15  -7.35
  4.5  -1.15  -7.35
 -4.5   0.58  -7.35
 -2.5   0.58  -7.35
  ⋮           
  3.5  -0.58   7.35
 -3.5   1.15   7.35
 -1.5   1.15   7.35
  0.5   1.15   7.35
  2.5   1.15   7.35
  4.5   1.15   7.35
 -2.5   2.89   7.35
 -0.5   2.89   7.35
  1.5   2.89   7.35
  3.5   2.89   7.35
 -1.5   4.62   7.35
  0.5   4.62   7.35

In [91]:
# Generating Struct
CUDA.@time Agg = Aggregate(Neighbor(50, 12), PositionCell(X));

  0.129822 seconds (345.44 k CPU allocations: 20.699 MiB) (12 GPU allocations: 4.813 MiB, 0.07% memmgmt time)


## run_event -> one_aggregate

In [92]:
# # Simulation One Aggregate
# CUDA.@time one_aggregate(true, parameters, Cubic(1.0, 2.0, 3.0), Agg)

## run_event -> fusion

In [93]:
# Simulating Two Aggregate
CUDA.@time fusion(true, parameters, Cubic(100, 2.0, 3.0), Agg)

Finding Equilibrium in two Aggregate


Progress:  19%|████▋                    |  ETA: 0:03:169:41

LoadError: TaskFailedException

[91m    nested task error: [39mInterruptException:
    Stacktrace:
     [1] [0m[1miolock_end[22m
    [90m   @ [39m[90m./[39m[90;4mlibuv.jl:49[0m[90m [inlined][39m
     [2] [0m[1mclose[22m
    [90m   @ [39m[90m./[39m[90;4masyncevent.jl:146[0m[90m [inlined][39m
     [3] [0m[1mmacro expansion[22m
    [90m   @ [39m[90m~/.julia/packages/CUDA/DfvRa/lib/cudadrv/[39m[90;4mstream.jl:184[0m[90m [inlined][39m
     [4] [0m[1m(::CUDA.var"#15#18"{Timer, Base.Event})[22m[0m[1m([22m[0m[1m)[22m
    [90m   @ [39m[35mCUDA[39m [90m./[39m[90;4mthreadingconstructs.jl:169[0m